In [ ]:
!pip install -Uqq --upgrade pip
!pip install -Uqq auto-gptq
!pip install -Uqq transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

In [ ]:
model_name_or_path = "TheBloke/Luna-AI-Llama2-Uncensored-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=False,
        device="cuda:0",
        use_triton=False,
        quantize_config=None)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def generate_summary(paragraph):
    summarize_prompt = "please summarize the information given in the entire context in a short and crisp manner. Please follow the rules: 1.Make sure all important details are covered. 2.Make sure all parts of the context are covered. 3.Restrict your answer only to the given context. 4.Never mention the context. 5.Summarize in formal third person format. 6.Summarize into one and only single paragraph and dont use bullet points or numbering and do not use headings"
    summarize_context = paragraph
    prompt_template=f'''A chat between a curious user and an artificial intelligence assistant. The assistant uses the given context to summarize the users context in the given format of a single paragraph and restricts its answer only to the given context.

    USER: {summarize_prompt}
    CONTEXT: {summarize_context}
    FORMAT: 1.Single paragraph 2.Only body and no heading. 3.No bullet points
    ASSISTANT:
    '''

    input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.45, max_new_tokens=512, do_sample=True)
    s = tokenizer.decode(output[0])
    summary = s[s.find('ASSISTANT:')+10:]

    print(summary)

In [ ]:
# Summary
generate_summary("HAN 08-ch01-001-038-9780123814791 2011/6/1 3:12 Page 1 #1 1Introduction This book is an introduction to the young and fast-growing ﬁeld of data mining (also known asknowledge discovery from data, or KDD for short). The book focuses on fundamental data mining concepts and techniques for discovering interesting patterns from data in various applications. In particular, we emphasize prominent techniques for developing effective, efﬁcient, and scalable data mining tools. This chapter is organized as follows. In Section 1.1, you will learn why data mining is in high demand and how it is part of the natural evolution of information technology. Section 1.2 deﬁnes data mining with respect to the knowledge discovery process. Next, you will learn about data mining from many aspects, such as the kinds of data that can be mined (Section 1.3), the kinds of knowledge to be mined (Section 1.4), the kinds of technologies to be used (Section 1.5), and targeted applications (Section 1.6). In this way, you will gain a multidimensional view of data mining. Finally, Section 1.7 outlines major data mining research and development issues. 1.1Why Data Mining? Necessity, who is the mother of invention. – Plato We live in a world where vast amounts of data are collected daily. Analyzing such data is an important need. Section 1.1.1 looks at how data mining can meet this need by providing tools to discover knowledge from data. In Section 1.1.2, we observe how data mining can be viewed as a result of the natural evolution of information technology. 1.1.1 Moving toward the Information Age “We are living in the information age ” is a popular saying; however, we are actually living in the data age. T erabytes or petabytes1of data pour into our computer networks, the World Wide Web (WWW), and various data storage devices every day from business, 1A petabyte is a unit of information or computer storage equal to 1 quadrillion bytes, or a thousand terabytes, or 1 million gigabytes. c 2012 Elsevier Inc. All rights reserved.Data Mining: Concepts and T echniques 1HAN 08-ch01-001-038-9780123814791 2011/6/1 3:12 Page 2 #2 2 Chapter 1 Introduction society, science and engineering, medicine, and almost every other aspect of daily life. This explosive growth of available data volume is a result of the computerization of our society and the fast development of powerful data collection and storage tools. Businesses worldwide generate gigantic data sets, including sales transactions, stock trading records, product descriptions, sales promotions, company proﬁles and perfor- mance, and customer feedback. For example, large stores, such as Wal-Mart, handle hundreds of millions of transactions per week at thousands of branches around the world. Scientiﬁc and engineering practices generate high orders of petabytes of data in a continuous manner, from remote sensing, process measuring, scientiﬁc experiments, system performance, engineering observations, and environment surveillance. Global backbone telecommunication networks carry tens of petabytes of data trafﬁc every day. The medical and health industry generates tremendous amounts of data from medical records, patient monitoring, and medical imaging. Billions of Web searches supported by search engines process tens of petabytes of data daily. Communities and social media have become increasingly important data sources, producing digital pic- tures and videos, blogs, Web communities, and various kinds of social networks. The list of sources that generate huge amounts of data is endless. This explosively growing, widely available, and gigantic body of data makes our time truly the data age. Powerful and versatile tools are badly needed to automatically uncover valuable information from the tremendous amounts of data and to transform such data into organized knowledge. This necessity has led to the birth of data mining. The ﬁeld is young, dynamic, and promising. Data mining has and will continue to make great strides in our journey from the data age toward the coming information age. Example 1.1 Data mining turns a large collection of data into knowledge. A search engine (e.g., Google) receives hundreds of millions of queries every day. Each query can be viewed as a transaction where the user describes her or his information need. What novel and useful knowledge can a search engine learn from such a huge collection of queries col- lected from users over time? Interestingly, some patterns found in user search queries can disclose invaluable knowledge that cannot be obtained by reading individual data items alone. For example, Google’s Flu Trends uses speciﬁc search terms as indicators of ﬂu activity. It found a close relationship between the number of people who search for ﬂu-related information and the number of people who actually have ﬂu symptoms. A pattern emerges when all of the search queries related to ﬂu are aggregated. Using aggre- gated Google search data, Flu Trends can estimate ﬂu activity up to two weeks faster than traditional systems can.2This example shows how data mining can turn a large collection of data into knowledge that can help meet a current global challenge. 1.1.2 Data Mining as the Evolution of Information T echnology Data mining can be viewed as a result of the natural evolution of information tech- nology. The database and data management industry evolved in the development of 2This is reported in [GMPC09].HAN 08-ch01-001-038-9780123814791 2011/6/1 3:12 Page 3 #3 1.1 Why Data Mining? 3 Data Collection and Database Creation (1960s and earlier) Primitive file processing Database Management Systems (1970s to early 1980s) Hierarchical and network database systems Relational database systems Data modeling: entity-relationship models, etc. Indexing and accessing methods Query languages: SQL, etc. User interfaces, forms, and reports Query processing and optimization Transactions, concurrency control, and recovery Online transaction processing (OLTP) Advanced Database Systems (mid-1980s to present) Advanced data models: extended-relational, object relational, deductive, etc. Managing complex data: spatial, temporal, multimedia, sequence and structured, scientific, engineering, moving objects, etc. Data streams and cyber-physical data systems Web-based databases (XML, semantic web) Managing uncertain data and data cleaning Integration of heterogeneous sources Text database systems and integration with information retrieval Extremely large data management Database system tuning and adaptive systems Advanced queries: ranking, skyline, etc. Cloud computing and parallel data processing Issues of data privacy and securityAdvanced Data Analysis (late- 1980s to present) Data warehouse and OLAP Data mining and knowledge discovery: classification, clustering, outlier analysis, association and correlation, comparative summary, discrimination analysis, pattern discovery, trend and deviation analysis, etc. Mining complex types of data: streams, sequence, text, spatial, temporal, multimedia, Web, networks, etc. Data mining applications: business, society, retail, banking, telecommunications, science and engineering, blogs, daily life, etc. Data mining and society: invisible data mining, privacy-preserving data mining, mining social and information networks, recommender systems, etc. Future Generation of Information Systems (Present to future) Figure 1.1 The evolution of database system technology. several critical functionalities (Figure 1.1): data collection and database creation, data management (including data storage and retrieval and database transaction processing), and advanced data analysis (involving data warehousing and data mining). The early development of data collection and database creation mechanisms served as a prerequi- site for the later development of effective mechanisms for data storage and retrieval, as well as query and transaction processing. Nowadays numerous database systems offer query and transaction processing as common practice. Advanced data analysis has naturally become the next step.HAN 08-ch01-001-038-9780123814791 2011/6/1 3:12 Page 4 #4 4 Chapter 1 Introduction Since the 1960s, database and information technology has evolved systematically from primitive ﬁle processing systems to sophisticated and powerful database systems. The research and development in database systems since the 1970s progressed from early hierarchical and network database systems to relational database systems (where data are stored in relational table structures; see Section 1.3.1), data modeling tools, and indexing and accessing methods. In addition, users gained convenient and ﬂexible data access through query languages, user interfaces, query optimization, and transac- tion management. Efﬁcient methods for")



*** Summary:

    1.1 Why Data Mining?
Necessity, who is the mother of invention. - Plato
We live in a world where vast amounts of data are collected daily. Analyzing such data is an important need. Section 1.1.1 looks at how data mining can meet this need by providing tools to discover knowledge from data. In Section 1.1.2, we observe how data mining can be viewed as a result of the natural evolution of information technology.
The database and data management industry evolved in the development of primitive file processing database management systems (1970s to early 1980s) hierarchical and network database systems relational database systems data modeling: entity-relationship models, etc. indexing and accessing methods query languages: SQL, etc. user interfaces, forms, and reports query processing and optimization transactions, concurrency control, and recovery online transaction processing (OLTP) advanced database systems (mid-1980s to present) advanced data models: extended-relati

In [ ]:
# Questions
prompt = "please Generate only 3 insightful, properly phrased questions and their answers from the given context. Ensure that the response is in the following format: '<question>?: <answer>'. it is critical that the format is followed. also DO NOT mention context"

context = summary

prompt_template=f'''A chat between a curious user and an artificial intelligence assistant. The assistant uses the given context to answer all of the users queries. Restrict your answer only to the given context. the user requires his answers in a QUESTION: ANSWER: format

USER: {prompt}
CONTEXT: {context}
ASSISTANT:
'''

print("\n\n*** Questions:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.5, max_new_tokens=512, do_sample=True)
q = tokenizer.decode(output[0])
questions = q[q.find('ASSISTANT:')+10:]

print(questions)



*** Questions:

1. What is the most popular sport in the world? 
: Soccer (football)
2. What is the largest organ in the human body? 
: Liver
3. What is the most widely spoken language in the world? 
: Mandarin Chinese</s>


In [ ]:
# Terms
prompt = "please make a List of 5 the important important terms and their definitions from the given context. Ensure that the response is in the following format '<term>: <meaning>'. It is critical that the format is followed. also DO NOT mention context. try to be as concise as possible"

context = summary

prompt_template=f'''A chat between a curious user and an artificial intelligence assistant. The assistant uses the given context to answer all of the users queries. Restrict your answer only to the given context. the user requires his response in a <term>: <meaning>:\n format

USER: {prompt}
CONTEXT: {context}
ASSISTANT:
'''

print("\n\n*** Terms:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.5, max_new_tokens=512, do_sample=True)
q = tokenizer.decode(output[0])
questions = q[q.find('ASSISTANT:')+10:]

print(questions)



*** Terms:

1. Context: A set of circumstances or background information that helps to interpret or understand something.
2. Context: A particular time or place in which something happens or is done.
3. Context: A set of assumptions, beliefs, or values that influences the way a person thinks or behaves.
4. Context: A set of rules or standards that govern behavior in a particular situation or organization.
5. Context: A set of factors or variables that influence the outcome of a situation or event.</s>
